## Get The text from the PDF

In [19]:
%pip install pdfplumber pytesseract pdf2image


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [20]:
import pdfplumber
import pytesseract
from pdf2image import convert_from_path

In [21]:
def extract_text_from_pdf(Sahil_resume):
    text = ""
    try:
        # Try direct text extraction
        with pdfplumber.open(Sahil_resume) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text

        if text.strip():
            return text.strip()
    except Exception as e:
        print(f"Direct text extraction failed: {e}")

    # Fallback to OCR for image-based PDFs
    print("Falling back to OCR for image-based PDF.")
    try:
        images = convert_from_path(Sahil_resume)
        for image in images:
            page_text = pytesseract.image_to_string(image)
            text += page_text + "\n"
    except Exception as e:
        print(f"OCR failed: {e}")

    return text.strip()

In [22]:
pdf_path = "Sahil_resumef"
resume_text = extract_text_from_pdf(pdf_path)

print("\nExtracted Text from PDF:")
print(resume_text)

Direct text extraction failed: [Errno 2] No such file or directory: 'Sahil_resumef'
Falling back to OCR for image-based PDF.
OCR failed: Unable to get page count. Is poppler installed and in PATH?

Extracted Text from PDF:



## Set Google GenerativeAI Api Key

In [23]:
%pip install google.generativeai python-dotenv


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [24]:
# import google.generativeai as genai
# import os
# from dotenv import load_dotenv

# load_dotenv()
# genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
# model = genai.GenerativeModel("gemini-1.5-flash")

import google.generativeai as genai
import os
from dotenv import load_dotenv

load_dotenv()  # Load environment variables from .env file
api_key = os.getenv("GOOGLE_API_KEY")
if not api_key:
    raise ValueError("API key not found. Please check your .env file.")

genai.configure(api_key=api_key)  # Set the API key for the client
model = genai.GenerativeModel("gemini-1.5-flash")



In [25]:
response = model.generate_content("What is the capital of India?")


In [26]:
print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "The capital of India is **New Delhi**.\n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -0.002600635588169098
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 7,
        "candidates_token_count": 10,
        "total_token_count": 17
      },
      "model_version": "gemini-1.5-flash"
    }),
)


In [27]:
print(response.text)

The capital of India is **New Delhi**.



## Resume Analysis

In [28]:
def analyze_resume(resume_text, job_description=None  ):
    if not resume_text:
        return {"error": "Resume text is required for analysis."}
    
    model = genai.GenerativeModel("gemini-1.5-flash")
    
    base_prompt = f"""
    You are an experienced HR with Technical Experience in the field of any one job role from Data Science, Data Analyst, DevOPS, Machine Learning Engineer, Prompt Engineer, AI Engineer, Full Stack Web Development, Big Data Engineering, Marketing Analyst, Human Resource Manager, Software Developer your task is to review the provided resume.
    Please share your professional evaluation on whether the candidate's profile aligns with the role.ALso mention Skills he already have and siggest some skills to imorve his resume , alos suggest some course he might take to improve the skills.Highlight the strengths and weaknesses.

    Resume:
    {resume_text}
    """

    if job_description:
        base_prompt += f"""
        Additionally, compare this resume to the following job description:
        
        Job Description:
        {job_description}
        
        Highlight the strengths and weaknesses of the applicant in relation to the specified job requirements.
        Hey Act Like a skilled or very experience ATS(Application Tracking System)
        with a deep understanding of tech field,software engineering,data science ,data analyst
        and big data engineer. Your task is to evaluate the resume based on the given job description.
        You must consider the job market is very competitive and you should provide 
        best assistance for improving thr resumes. Assign the percentage Matching based 
        on Jd and
        the missing keywords with high accuracy
        
        {{"JD Match":"%"\n,"MissingKeywords:[]"\n,"Profile Summary":""}}
        """

    response = model.generate_content(base_prompt)

    analysis = response.text.strip()
    return analysis

In [30]:
print(analyze_resume(resume_text))


{'error': 'Resume text is required for analysis.'}
